# Provider Packages 📦

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Anthropic
### General hosting via Anthropic
Note that it will **by default take the enviroment variable ANTHROPIC_API_KEY** as API key

- Python SDK Docs: https://github.com/anthropics/anthropic-sdk-python

#### NORMAL CALL

In [2]:
import anthropic

client = anthropic.Anthropic()
message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
message.model_dump()

{'id': 'msg_01CXc9ih6n3YciN89iMKTp2h',
 'content': [{'citations': None,
   'text': "Hello! It's nice to meet you. How are you doing today? Is there anything I can help you with or anything you'd like to chat about?",
   'type': 'text'}],
 'model': 'claude-sonnet-4-20250514',
 'role': 'assistant',
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'type': 'message',
 'usage': {'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'input_tokens': 10,
  'output_tokens': 35,
  'server_tool_use': None,
  'service_tier': 'standard'}}

#### TOOL USE (FUNCTION CALLING)

**Note you can also do multiple tool calls at once:**

In this case, Claude will most likely try to use two separate tools, one at a time — get_weather and then get_time — in order to fully answer the user’s question. However, it will also occasionally output two tool_use blocks at once, particularly if they are not dependent on each other. You would need to execute each tool and return their results in separate tool_result blocks within a single user message.

Note that for AnthropicVertex tool calling works exacly the same

In [3]:
# Single tool call example

messages=[{"role": "user", "content": "What's the weather like in San Francisco?"}]
tools=[
        {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        }
    ]

response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    tools=tools,
    messages=messages,
)

# Append the tool call to the message

messages.append({
    "role": "assistant",
    "content": response.model_dump()['content']
})
messages

[{'role': 'user', 'content': "What's the weather like in San Francisco?"},
 {'role': 'assistant',
  'content': [{'citations': None,
    'text': "I'll check the current weather in San Francisco for you.",
    'type': 'text'},
   {'id': 'toolu_012dBMeedTqYNQfGL9Yj3Cxf',
    'input': {'location': 'San Francisco, CA'},
    'name': 'get_weather',
    'type': 'tool_use'}]}]

In [4]:
# Let's say we execute the tool call
# Import we pass back the tool use id and the content

response_message = {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": "toolu_017WtHBKTNxjdu3qEBQauXDZ", # from the API response
                    "content": "65 degrees" # from running your tool
                }
            ]
        }

messages.append(response_message)
response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    tools=tools,
    messages=messages)

response.model_dump()

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.2.content.0: unexpected `tool_use_id` found in `tool_result` blocks: toolu_017WtHBKTNxjdu3qEBQauXDZ. Each `tool_result` block must have a corresponding `tool_use` block in the previous message.'}}

### Hosting via VertexAI (Google Cloud)
Here it's important to always specify the project ID from your Google Cloud project and the region where you want to model from what region you want to run inferance on.

- Check the [Vertex AI Locations](https://cloud.google.com/vertex-ai/docs/general/locations) for more information about the specifc regions.
- The Python SDK information: https://docs.anthropic.com/en/api/claude-on-vertex-ai
- Tool calling and other functions are the same as for the base Anthropic implementation

#### NORMAL CALL

In [ ]:
from anthropic import AnthropicVertex

project_id = os.getenv("GCP_PROJECT_ID")
# Where the model is running
region = os.getenv("GCP_LOCATION")
client = AnthropicVertex(project_id=project_id, region=region)

response = client.messages.create(
    model="claude-sonnet-4@20250514",
    max_tokens=100,
    messages=[
        {
            "role": "user",
            "content": "Hey Claude!",
        }
    ],
)
response.model_dump()

## OpenAI
### General hosting via OpenAI
Note that **it will take by default the enviroment variable OPENAI_API_KEY** as API KEY

- [Python SDK Docs](https://platform.openai.com/docs/libraries?language=python)

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

response.model_dump()

#### TOOL USE (FUNCTION CALLING)

In [ ]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False # This parameter is not available for both Anthropic and Google
    }
}]

messages = [
    {"role": "user", "content": "What is the weather like in Paris today?"}
]

response = client.responses.create(
    model="gpt-4.1",
    input=messages,
    tools=tools
)

# Note that we did not assign a role for the tool arguments provided by OpenAI
messages.extend(response.model_dump()['output'])
messages

In [ ]:
response = {
    "type": "function_call_output",
    "call_id": 'call_mtIdLDOVjfxWgjCw7eGi5WmS', # from the API response
    "output": "The weather in Paris is sunny with a temperature of 20 degrees Celsius."
}

messages.append(response)
response = client.responses.create(
    model="gpt-4.1",
    input=messages,
    tools=tools
)

response.model_dump()['output']

### Hosting via Azure

Check the documentation [here](https://learn.microsoft.com/en-gb/azure/ai-services/openai/supported-languages?tabs=dotnet-secure%2Csecure%2Cpython-key%2Ccommand&pivots=programming-language-pythong)

Note that not all models are available in every region, most EU models are available in **swedencentral** and US models in **eastus2**
Also remember to deploy the model you want to use on https://ai.azure.com/

- Azure OpenAI uses the same principles for calling functions as the base OpenAI Implementation

In [ ]:
from openai import AzureOpenAI

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2025-01-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

completion = client.chat.completions.create(
    model="gpt-4.1",  # e.g. gpt-35-instant
    messages=[
        {
            "role": "user",
            "content": "How do I output all files in a directory using Python?",
        },
    ],
)


## Google
### General Hosting (via AI Studio)

- Check the documentation on GitHub [here](https://github.com/googleapis/python-genai).

#### NORMAL CALL

In [ ]:
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

response = client.models.generate_content(
    model='gemini-2.0-flash-001', contents='Why is the sky blue?'
)
response.model_dump()

#### TOOL USE / FUNCTION CALLING

In [ ]:
from google.genai import types

tools = [{
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        }
}]

messages =  [
    types.Content(
        role="user", parts=[types.Part(text="What is the weather like in Paris today?")]
    )
]

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
tools = types.Tool(function_declarations=tools)
config = types.GenerateContentConfig(tools=[tools])


# Send request with function declarations
response = client.models.generate_content(
     model="gemini-2.0-flash",
     contents=messages,
     config=config,
)

response.model_dump()

In [ ]:
messages.append(response.candidates[0].content)

# Create a function response part
function_response_part = types.Part.from_function_response(
    name="get_weather",
    response={"result": "20 degrees Celsius and sunny"},
)

messages.append(types.Content(role="user", parts=[function_response_part]))

final_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=messages,
    config=config,
)

final_response.model_dump()['candidates'][0]

### Hosting via VertexAI
We do this using the same package as before but we now set the project id and location

In [ ]:
from google import genai

# Only run this block for Vertex AI API
client = genai.Client(
    vertexai=True, project=os.environ["GCP_PROJECT_ID"], location=os.environ["GCP_LOCATION"]
)

response = client.models.generate_content(
    model='gemini-2.0-flash-001', contents='Why is the sky blue?'
)
response